In [29]:
import warnings 
from typing import List, Dict, Tuple, Callable, Union
from math import pi
from itertools import batched
import pennylane as qml
import pennylane.numpy as np
from scipy.optimize import minimize
from qiskit.visualization import plot_histogram
from tqdm import tqdm

from data import  relations, starting_point, encode_angle, labels, labels_index, nodes_index_relations_split
  
  
warnings.filterwarnings('ignore') 

![graph](./assets/graph.png)

In [8]:
steps = n_qubits = len(labels)-1 # at each step, the algorithm can choose between all different nodes except A (once it's starting node)
qubits_iter = range(n_qubits)
dev = qml.device('qiskit.aer', wires=n_qubits, shots=1000)

In [33]:
type Model = Callable[List[float], float, Union[qml.Hamiltonian, None], bool]
type Distances = List[float]
type Weights = List[float]
type DistancesRelations = Dict[str,float]

In [34]:
@qml.qnode(dev)
def model(
    distances:Distances,
    current_path_size:float, 
    weights:Weights,
    hamiltonian:qml.Hamiltonian=None,
    debug:bool=True
) -> List[float]:

    """
        Generates the qml model using PennyLane + Qiskit.

        In total, the circuit contains 4 qubits in total but, except for the fist step, the algorithm itself uses only 3, once each step has a self 
        loop and a relation that returns to A, which are removed during the circuit setup.    
    """


    # it's required once the scipy's minimize function expects a one dimensional list but BasicEntanglerLayers a multidimensional
    weights = np.reshape(weights, (1,1,n_qubits))


    distances = [ distance + current_path_size for distance in distances ] # get new distances based on the current path
    farest_distance = max(distances)

    if(debug):
        print(f"Distances: {distances}")
        print(f"Farest Distance: {farest_distance}")

    for qubit,distance in enumerate(distances):
        qml.RY(encode_angle(distance, farest_distance), wires=qubit)
    qml.Barrier(wires=qubits_iter)
    qml.BasicEntanglerLayers(rotation=qml.RY, weights=weights, wires=qubits_iter)

    if(hamiltonian is not None):
        return qml.expval(hamiltonian)
    
    return qml.counts()

In [35]:
def get_step_distances(current_node:str, distances_relations:DistancesRelations) -> Tuple[List[str], List[float]]:
    """
        This function get all distances from inside the distances relations object and return it as a list.
        
        Based on the current node, the function get the distance if the node it leaves from is equal the current node, the nodes relation
        doesn't generate a loop and it ensures that the next node is different from the starting point, which was visited already.

        It also returns together with the distance the target node.  
    """
    next_nodes = []
    distances = []

    for connection, distance in distances_relations.items():
        if(connection[0] != current_node or connection[1] == starting_point or connection[0] == connection[1]):
            continue

        next_nodes.append(connection[1])
        distances.append(distance)

    return (next_nodes, distances)

In [80]:
def get_hamiltonian(distances:Distances) -> qml.Hamiltonian:
    """
        It must create a hamiltonian based on the distances following the Ising Spin Model.

        Also, it must add some Constraints to it, like one node per time. 
    """


    coeffs = [0.5 * distance for distance in distances]
    obs = [qml.PauliZ(qubit) for qubit in range(len(distances))]

    # constraints
    P = 30
    non_target_nodes = len(distances)-1
    coeffs.append(P * (non_target_nodes - 1))
    obs.append(qml.Identity(1))

    coeffs = [*coeffs, *[P for _ in range(len(distances))]]
    obs = [*obs, *[qml.PauliZ(qubit) for qubit in range(len(distances))]]

    return qml.Hamiltonian(coeffs=coeffs, observables=obs)


In [81]:
def get_next_node(nodes:List[str], already_visited:List[str]) -> str:
    random_node = lambda nodes: np.random.choice(nodes)
    
    next_node = random_node(nodes)
    while next_node in already_visited:
        next_node = random_node(nodes)

    return next_node

In [85]:
def train(weights:Weights, model:Model, relations:DistancesRelations) -> float:
    path = []
    total_expectation = 0
    path_size = 0

    current_node = starting_point

    for step in trange(steps):
        # evaluate
        next_nodes, distances = get_step_distances(current_node, relations)
        hamiltonian = get_hamiltonian(distances)
        print(hamiltonian)
        expectation_value = model(distances, path_size, weights, hamiltonian, debug=False)

        total_expectation += expectation_value

        # update values
        new_node = get_next_node(next_nodes, path)
        new_node_weight = distances[next_nodes.index(new_node)]
        
        path.append(new_node)
        path_size += new_node_weight

        current_node = new_node


    return total_expectation

In [86]:
weights = np.random.uniform(0, np.pi, size=n_qubits, requires_grad=True)
minimized = minimize(x0=weights, fun=train, args=(model, relations), method="COBYLA")

 25%|██▌       | 1/4 [00:00<00:00,  9.19it/s]

2.9 * Z(0) + 2.9 * Z(1) + 3.8 * Z(2) + 4.4 * Z(3) + 60 * I(1) + 30 * Z(0) + 30 * Z(1) + 30 * Z(2) + 30 * Z(3)
0.85 * Z(0) + 1.75 * Z(1) + 2.35 * Z(2) + 30 * I(1) + 30 * Z(0) + 30 * Z(1) + 30 * Z(2)
2.0 * Z(0) + 1.7 * Z(1) + 0.7 * Z(2) + 30 * I(1) + 30 * Z(0) + 30 * Z(1) + 30 * Z(2)


100%|██████████| 4/4 [00:00<00:00, 12.35it/s]


2.55 * Z(0) + 2.25 * Z(1) + 0.7 * Z(2) + 30 * I(1) + 30 * Z(0) + 30 * Z(1) + 30 * Z(2)


  0%|          | 0/4 [00:00<?, ?it/s]

...

 50%|█████     | 2/4 [00:00<00:00, 15.82it/s]

2.9 * Z(0) + 2.9 * Z(1) + 3.8 * Z(2) + 4.4 * Z(3) + 60 * I(1) + 30 * Z(0) + 30 * Z(1) + 30 * Z(2) + 30 * Z(3)
2.0 * Z(0) + 1.7 * Z(1) + 0.7 * Z(2) + 30 * I(1) + 30 * Z(0) + 30 * Z(1) + 30 * Z(2)
2.55 * Z(0) + 2.25 * Z(1) + 0.7 * Z(2) + 30 * I(1) + 30 * Z(0) + 30 * Z(1) + 30 * Z(2)
1.2 * Z(0) + 2.15 * Z(1) + 2.15 * Z(2) + 30 * I(1) + 30 * Z(0) + 30 * Z(1) + 30 * Z(2)


 50%|█████     | 2/4 [00:00<00:00, 15.75it/s]

2.9 * Z(0) + 2.9 * Z(1) + 3.8 * Z(2) + 4.4 * Z(3) + 60 * I(1) + 30 * Z(0) + 30 * Z(1) + 30 * Z(2) + 30 * Z(3)
0.85 * Z(0) + 1.75 * Z(1) + 2.35 * Z(2) + 30 * I(1) + 30 * Z(0) + 30 * Z(1) + 30 * Z(2)
2.0 * Z(0) + 1.7 * Z(1) + 0.7 * Z(2) + 30 * I(1) + 30 * Z(0) + 30 * Z(1) + 30 * Z(2)
1.2 * Z(0) + 2.15 * Z(1) + 2.15 * Z(2) + 30 * I(1) + 30 * Z(0) + 30 * Z(1) + 30 * Z(2)


100%|██████████| 4/4 [00:00<00:00, 15.20it/s]


In [75]:
def simulate_path(starting_node:str, relations:DistancesRelations, weights:Weights, model:Model):
    path = []
    path_size = 0

    current_node = starting_node

    for step in range(steps):
        next_nodes, distances = get_step_distances(current_node, relations)
        counts = model(distances, path_size,  weights)
        print(counts)
        
        

In [84]:
simulate_path(starting_point, relations, minimized.x, model)

Distances: [5.8, 5.8, 7.6, 8.8]
Farest Distance: 8.8
[{np.str_('0000'): np.int64(22), np.str_('0001'): np.int64(3), np.str_('0010'): np.int64(11), np.str_('0100'): np.int64(26), np.str_('0110'): np.int64(555), np.str_('1001'): np.int64(76), np.str_('1011'): np.int64(5), np.str_('1101'): np.int64(111), np.str_('1111'): np.int64(191)}]
Distances: [5.8, 5.8, 7.6, 8.8]
Farest Distance: 8.8
[{np.str_('0000'): np.int64(25), np.str_('0001'): np.int64(2), np.str_('0010'): np.int64(10), np.str_('0100'): np.int64(30), np.str_('0110'): np.int64(536), np.str_('1000'): np.int64(2), np.str_('1001'): np.int64(76), np.str_('1010'): np.int64(3), np.str_('1011'): np.int64(1), np.str_('1101'): np.int64(138), np.str_('1110'): np.int64(1), np.str_('1111'): np.int64(176)}]
Distances: [5.8, 5.8, 7.6, 8.8]
Farest Distance: 8.8
[{np.str_('0000'): np.int64(23), np.str_('0001'): np.int64(2), np.str_('0010'): np.int64(12), np.str_('0100'): np.int64(27), np.str_('0110'): np.int64(596), np.str_('1001'): np.int64(66